In [1]:

from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords


tt = TweetTokenizer()
stopwords = set(stopwords.words('english')) # note: stopwords are all in lowercase

def tokenizing_english_xstopwords(tw_text):
    
#     text = re.sub(r"http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+", "URL", text)  # Replace urls with special token
#     text = text.replace("\'s", "")
#     text = text.replace("\'", "")
#     text = text.replace("n\'t", " n\'t")
#     text = text.replace("@", "")
#     text = text.replace("#", "")
#     text = text.replace("_", " ")
#     text = text.replace("-", " ")
#     text = text.replace("&amp;", "")
#     text = text.replace("&gt;", "")
#     text = text.replace("\"", "")
#     text = text.replace(".", "")
#     text = text.replace(",", "")
#     text = text.replace("(", "")
#     text = text.replace(")", "")
#     text = ' '.join(text.split())
    
    
    tw_token = tt.tokenize(tw_text)
    tw_token_lower = [w.lower() for w in tw_token]
    tw_token_lower_english = [w for w in tw_token_lower if w.isalpha()]
    tw_token_lower_english_xstopword = [w for w in tw_token_lower_english if not w in stopwords]
    return tw_token_lower_english_xstopword


In [105]:

import json
import pandas as pd


lines = []
with open('train.data.txt') as f:
    lines = f.readlines()
labels = []
with open('train.label.txt') as f:
    labels = f.readlines()
data_label = [lb.splitlines()[0] for lb in labels]
data_label_01 = []
for lb in data_label:
    if lb == 'rumour':
        data_label_01.append((0, 1))
    else:
        data_label_01.append((1, 0))


tw_path = 'data/train-data/'
col = ['tw_text_re', 'label0', 'label1']
inx = range(len(data_label))
data_train = pd.DataFrame(columns=col, index=inx)
# data_train = pd.DataFrame(columns=col)
# data_train['label'] = data_label
# display(data_train)

for i in range(len(lines)):
    line = lines[i]
    twtw = line.splitlines()[0]
    tw_list = str(twtw).split(',')
    
    tw_text_re = []
    for twid in tw_list:
        try:
            with open(tw_path+twid+'.json') as f:
                tw_object = json.load(f)
        except FileNotFoundError:
            continue
        text = tw_object['text']
        tw_tokenized = tokenizing_english_xstopwords(text)
        tw_text_re += tw_tokenized
    
    # data_train['tw_text_re'][i] = tw_text_re
    data_train['tw_text_re'][i] = ' '.join(map(str, tw_text_re))
    data_train['label0'][i] = data_label_01[i][0]
    data_train['label1'][i] = data_label_01[i][1]
    

data_train_drop = data_train.dropna().reset_index(drop=True)
display(data_train_drop)



,tw_text_re,label0,label1
0,regularly rinsing nose saline help prevent inf...,1,0
1,french police chief killed attack devastating ...,0,1
2,coronavirus disease covid advice public wear m...,1,0
3,ottawa police confirm multiple suspects shooti...,1,0
4,primary focus government alleviate suffering d...,1,0
...,...,...,...
1890,desperate ted cruz claims planned parenthood s...,0,1
1891,thoughts prayers enough pres obama speaks mass...,0,1
1892,police surrounded building suspected attackers...,1,0
1893,joseph smith translated gift power god come fo...,1,0


In [106]:
import numpy as np
import pandas as pd

label_arr = data_train_drop['label1'].to_numpy()
label_count_1 = np.count_nonzero(label_arr == 1)
label_count_0 = np.count_nonzero(label_arr == 0)
d = label_count_0 - label_count_1

data_train_drop_1 = data_train_drop[data_train_drop['label1'] == 1]
data_sample = data_train_drop_1.sample(n=d, replace=True, random_state=90042)
data_train_drop = pd.concat([data_train_drop, data_sample], ignore_index = True).reset_index()

data_train_drop = data_train_drop.drop(['index'], axis=1)
data_train_drop = data_train_drop.to_numpy()
np.random.shuffle(data_train_drop)
data_train_drop = pd.DataFrame(data_train_drop, columns=['tw_text_re', 'label0', 'label1'])
data_train_drop

,tw_text_re,label0,label1
0,new low obama spent money close natl parks shu...,0,1
1,news shots fired parliament hill follow develo...,1,0
2,worry ppl threat homeland man wearing body arm...,0,1
3,wash fruit vegetables time handling wash hands...,1,0
4,lit like rainbow awesome getting better light ...,1,0
...,...,...,...
2945,obama pattern mass shootings country parallel ...,0,1
2946,photo white house lit rainbow colors high gay ...,1,0
2947,covid virus transmitted areas hot humid climat...,1,0
2948,number ppl killed mass shootings obama tenure ...,0,1


In [107]:

label_arr = data_train_drop['label1'].to_numpy()
label_count_1 = np.count_nonzero(label_arr == 1)
label_count_0 = np.count_nonzero(label_arr == 0)
print(label_count_1)
print(label_count_0)


1475
1475


In [108]:
import json
import pandas as pd

# lines = []
with open('dev.data.txt') as f:
    lines = f.readlines()
# labels = []
with open('dev.label.txt') as f:
    labels = f.readlines()
data_label = [lb.splitlines()[0] for lb in labels]
data_label_01 = []
for lb in data_label:
    if lb == 'rumour':
        data_label_01.append((0, 1))
    else:
        data_label_01.append((1, 0))

tw_path = 'data/dev-data/'
col = ['tw_text_re', 'label0', 'label1']
inx = range(len(data_label))
data_dev = pd.DataFrame(columns=col, index=inx)
# data_dev = pd.DataFrame(columns=col)
# data_dev['label'] = data_label
# display(data_dev)

for i in range(len(lines)):
    line = lines[i]
    twtw = line.splitlines()[0]
    tw_list = str(twtw).split(',')

    tw_text_re = []
    for twid in tw_list:
        try:
            with open(tw_path + twid + '.json') as f:
                tw_object = json.load(f)
        except FileNotFoundError:
            continue
        text = tw_object['text']
        tw_tokenized = tokenizing_english_xstopwords(text)
        tw_text_re += tw_tokenized

    # data_dev['tw_text_re'][i] = tw_text_re
    data_dev['tw_text_re'][i] = ' '.join(map(str, tw_text_re))
    data_dev['label0'][i] = data_label_01[i][0]
    data_dev['label1'][i] = data_label_01[i][1]

data_dev_drop = data_dev.dropna().reset_index(drop=True)
display(data_dev_drop)



,tw_text_re,label0,label1
0,covid fact hand dryers effective killing new c...,1,0
1,expect result pending antibody test please may...,1,0
2,covid spread people catch covid others virus d...,1,0
3,every news outlet using headlines like antibio...,1,0
4,researcher encounter goliath birdeater largest...,1,0
...,...,...,...
627,cure covid however several ongoing clinical tr...,1,0
628,speculation arrested banksy debuts new mural b...,0,1
629,questions answered reply number get correct an...,1,0
630,operation ku klux klan never stopped watching ...,0,1


In [109]:
import numpy as np
import pandas as pd

label_arr = data_dev_drop['label1'].to_numpy()
label_count_1 = np.count_nonzero(label_arr == 1)
label_count_0 = np.count_nonzero(label_arr == 0)
d = label_count_0 - label_count_1

data_dev_drop_1 = data_dev_drop[data_dev_drop['label1'] == 1]
data_sample = data_dev_drop_1.sample(n=d, replace=True, random_state=90042)
data_dev_drop = pd.concat([data_dev_drop, data_sample], ignore_index = True).reset_index()

data_dev_drop = data_dev_drop.drop(['index'], axis=1)
data_dev_drop = data_dev_drop.to_numpy()
np.random.shuffle(data_dev_drop)
data_dev_drop = pd.DataFrame(data_dev_drop, columns=['tw_text_re', 'label0', 'label1'])
data_dev_drop

,tw_text_re,label0,label1
0,live citizens assembly talk citizens assembly ...,1,0
1,world health organization insists new coronavi...,1,0
2,poll lead shrinks edge gop vanishes believe po...,0,1
3,symptoms covid covid airborne answers biggest ...,1,0
4,,1,0
...,...,...,...
981,covid virus transmitted areas hot humid climat...,1,0
982,flu much bigger threat coronavirus written th ...,0,1
983,macklemore announced today joined isis lol mac...,0,1
984,exquisite good chop translucent butterfly beau...,0,1


In [110]:

label_arr = data_dev_drop['label1'].to_numpy()
label_count_1 = np.count_nonzero(label_arr == 1)
label_count_0 = np.count_nonzero(label_arr == 0)
print(label_count_1)
print(label_count_0)


493
493


In [4]:
import json
import pandas as pd

# lines = []
with open('test.data.txt') as f:
    lines = f.readlines()


tw_path = 'data/tweet-objects/'
col = ['tw_text_re']
inx = range(len(data_label))
data_test = pd.DataFrame(columns=col, index=inx)
# display(data_test)

for i in range(len(lines)):
    line = lines[i]
    twtw = line.splitlines()[0]
    tw_list = str(twtw).split(',')

    tw_text_re = []
    for twid in tw_list:
        try:
            with open(tw_path + twid + '.json') as f:
                tw_object = json.load(f)
        except FileNotFoundError:
            continue
        text = tw_object['text']
        tw_tokenized = tokenizing_english_xstopwords(text)
        tw_text_re += tw_tokenized

    # data_test['tw_text_re'][i] = tw_text_re
    data_test['tw_text_re'][i] = ' '.join(map(str, tw_text_re))

data_test_drop = data_test.dropna().reset_index(drop=True)
display(data_test_drop)



,tw_text_re
0,covid spread thanks wcco station trust media p...
1,hate keep saying capitalism implode without vi...
2,q covid influenza viruses different q covid in...
3,una de les q coronaviruses de la pàgina web de...
4,absolutely blame politicians whoever else invo...
...,...
553,man dies disease situação meio que irônica man...
554,holy shit doritos flavored mountain dew lost a...
555,banksy account joins cartoonists support je su...
556,q members international heal q members adviser...


In [111]:
import numpy as np

x_train, y_train = data_train_drop.loc[:, 'tw_text_re'], data_train_drop.loc[:, 'label0':'label1']
x_dev, y_dev = data_dev_drop.loc[:, 'tw_text_re'], data_dev_drop.loc[:, 'label0':'label1']
x_test = data_test_drop.loc[:, 'tw_text_re']

x_train = np.array(x_train)#.astype(np.float32)
x_dev = np.array(x_dev)#.astype(np.float32)
x_test = np.array(x_test)
y_train = np.array(y_train).astype(np.float32)
y_dev = np.array(y_dev).astype(np.float32)


In [112]:
x_dev.shape

(986,)

In [113]:
y_dev.shape

(986, 2)

In [114]:
from keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(oov_token="<UNK>")
tokenizer.fit_on_texts(x_train)

x_train = tokenizer.texts_to_matrix(x_train, mode="count") #BOW representation
x_dev = tokenizer.texts_to_matrix(x_dev, mode="count") #BOW representation
x_test = tokenizer.texts_to_matrix(x_test, mode="count") #BOW representation

vocab_size = x_train.shape[1]
print("Vocab size =", vocab_size)
# print(x_train[0])

Vocab size = 17221


In [124]:
from keras.models import Sequential
from keras import layers

#model definition
model = Sequential(name="feedforward-bow-input")
# model.add(layers.Dense(10, input_dim=vocab_size, activation='relu'))
# model.add(layers.Dense(1, activation='sigmoid'))
model.add(layers.Dense(1000, input_dim=vocab_size, activation='elu'))
# model.add(layers.Dropout(0.2))
model.add(layers.Dense(1000, activation='elu'))
# model.add(layers.Dense(10, activation='elu'))
model.add(layers.Dropout(0.2))
model.add(layers.Dense(2, activation='softmax'))

#since it's a binary classification problem, we use a binary cross entropy loss here binary_crossentropy
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

#training

model.fit(x_train, y_train, epochs=20, verbose=True, validation_data=(x_dev, y_dev), batch_size=32)
# model.fit(x_train, y_train, epochs=20, verbose=True, batch_size=16)

loss, accuracy = model.evaluate(x_dev, y_dev, verbose=False)
print("\nTesting Accuracy:  {:.4f}".format(accuracy))


Model: "feedforward-bow-input"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_44 (Dense)             (None, 1000)              17222000  
_________________________________________________________________
dense_45 (Dense)             (None, 1000)              1001000   
_________________________________________________________________
dropout_14 (Dropout)         (None, 1000)              0         
_________________________________________________________________
dense_46 (Dense)             (None, 2)                 2002      
Total params: 18,225,002
Trainable params: 18,225,002
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20
93/93 [==============================] - 13s 134ms/step - loss: 0.4598 - accuracy: 0.8213 - val_loss: 0.5466 - val_accuracy: 0.8073
Epoch 2/20
93/93 [==============================] - 11s 118ms/step - loss: 0.0414 - accu

In [125]:

yhat = model.predict(x_test)
yhat


array([[9.9717879e-01, 2.8211770e-03],
       [1.0000000e+00, 2.9298658e-08],
       [9.9981779e-01, 1.8219123e-04],
       ...,
       [5.3621910e-04, 9.9946374e-01],
       [9.9999666e-01, 3.3396727e-06],
       [9.9999988e-01, 1.1146662e-07]], dtype=float32)

In [126]:
yhat.shape

(558, 2)

In [127]:
# result = yhat.copy()
# result[result>=0.5] = 1
# result[result<0.5] = 0
# result

In [128]:
import pandas as pd

df_result = pd.DataFrame(columns=['Id','Predicted'], index=range(len(yhat)))
for i in range(len(yhat)):
    df_result['Id'][i] = i
    if yhat[i][0] > yhat[i][1]:
        df_result['Predicted'][i] = 0
    else:
        df_result['Predicted'][i] = 1


df_result


,Id,Predicted
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0
...,...,...
553,553,0
554,554,0
555,555,1
556,556,0


In [129]:
df_result.to_csv('result03.csv', index=False)